In [1]:
#%%
import pyglet
import pymunk as pm
from pymunk.pyglet_util import DrawOptions
import time
import numpy as np
from dataclasses import dataclass
from typing import Any
import math
from joblib import Parallel, delayed
from numba import njit
from numba.experimental import jitclass
from copy import deepcopy
from scipy.stats import norm


class Cell:
    def __init__(self,
    length, 
    width, 
    x_pos, 
    y_pos,
    angle = 0, 
    resolution = 30, 
    model = "adder", 
    food_conc = 15, 
    mass = 0.000001, 
    friction = 0, 
    pm_object = 0, 
    division_threshold = 20*10,
    mother = None):
        self.length = length
        self.width = width
        self.x_pos = x_pos
        self.y_pos = y_pos
        self.angle = angle
        self.resolution = resolution
        self.model = model
        self.food_conc = food_conc
        self.mass = mass
        self.friction = friction
        self.pm_object = self.initialise_pm_object()
        self.division_threshold = division_threshold
        self.mother = mother
        self.lysis_p = 0
        
    
    def get_cell_vertices_for_draw(self, cell_length, cell_width, resolution):
    
        def circ(theta, start, radius):
            y = radius * np.cos(theta) +radius
            x = radius * np.sin(theta) + start + radius
            return x, y


        def wall(radius, start, end, t_or_b):
            wall_x = np.linspace(start, end, num = resolution)
            wall_y = np.ones(resolution)*radius * t_or_b +radius
            return wall_x, wall_y

        cell_width = cell_width/2
        cell_length = cell_length - cell_width
        left_wall = circ(np.linspace(np.pi,2*np.pi, num=resolution), 0, cell_width)
        top_wall_xy = wall(cell_width, cell_width, cell_length, 1)
        bottom_wall_xy = wall(cell_width, cell_width, cell_length, -1)
        right_wall = circ(np.linspace(0,np.pi, num=resolution), cell_length - cell_width, cell_width)
        return [[left_wall[0][x] - cell_length/2, left_wall[1][x] - cell_width/2] for x in reversed(range(len(left_wall[0])))] + \
                [[bottom_wall_xy[0][x] - cell_length/2, bottom_wall_xy[1][x]- cell_width/2] for x in (range(len(bottom_wall_xy[0])))] + \
                [[right_wall[0][x] - cell_length/2, right_wall[1][x]- cell_width/2] for x in reversed(range(len(right_wall[0])))] + \
                [[top_wall_xy[0][x] - cell_length/2, top_wall_xy[1][x]- cell_width/2] for x in reversed(range(len(top_wall_xy[0])))]

    
    def rotate(self, origin, point, angle):
        """
        Rotate a point counterclockwise by a given angle around a given origin.

        The angle should be given in radians.
        """
        ox, oy = origin
        px, py = point

        qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
        qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
        return qx, qy


    def vertices(self):
        points = np.array(self.get_cell_vertices_for_draw(self.length, self.width, self.resolution))
        points[:,0] = points[:,0] + self.x_pos
        points[:,1] = points[:,1] + self.y_pos

        rotated = np.zeros((len(points),2))
        for x in range(len(points)):
            rotated[x] = self.rotate((self.x_pos, self.y_pos), (points[x][0],points[x][1]), self.angle)

        return rotated

    def growth_rate(self):
        return self.food_conc


    def growth(self, dt):
        if self.model == "adder":
            self.length = self.length + dt * self.growth_rate()
            self.is_dividing()
            self.x_pos, self.y_pos = self.pm_object[1].position[0], self.pm_object[1].position[1]
            #self.angle =  self.pm_object[1].angle
    
    def initialise_pm_object(self):
        _poly_vertices = [tuple(vertex) for vertex in self.vertices().tolist()]
        _poly = pm.Poly(None, _poly_vertices)

        _poly.friction = self.friction
        _moment = pm.moment_for_poly(self.mass, _poly.get_vertices())
        _body = pm.Body(self.mass, _moment)
        _body._set_angle = self.angle
        _poly.body = _body
        
        _body.position = self.x_pos, self.y_pos
        
        return (_poly, _body)

    def update_pm_object(self):
        self.pm_object = self.initialise_pm_object()


    def is_dividing(self):
        if self.length > self.division_threshold * np.random.uniform(low = 0.9, high = 1.1):
            return 1
        if self.length < self.division_threshold:
            return 0
    def divide(self):
        self.length = self.length/2 * 0.98
        self.x_pos = self.x_pos - self.length/4 * np.cos(self.angle)
        self.y_pos = self.y_pos - self.length/4 * np.sin(self.angle)
        self.angle = np.random.uniform(low = self.angle - 2*np.pi*0.01, high = self.angle + 2*np.pi*0.01)
    def centroid(self):
        length = self.vertices().shape[0]
        sum_x = np.sum(self.vertices()[:, 0])
        sum_y = np.sum(self.vertices()[:, 1])
        return sum_x/length, sum_y/length

    def daughter_length(self):
        return self.length 
    def daughter_width(self):
        return np.random.uniform(low = self.width*0.99, high = self.width*1.01)
    def daughter_x_pos(self):
        return self.x_pos + self.length/2 * np.cos(self.angle)
    def daughter_y_pos(self):
        return self.y_pos + self.length/2 * np.sin(self.angle)
    def daughter_angle(self):
        return  np.random.uniform(low = self.angle - 2*np.pi*0.01, high = self.angle + 2*np.pi*0.01)


import pandas as pd
import pymunk
import numpy as np
def segment_creator(local_xy1, local_xy2,global_xy,thickness):
    segment_body = pymunk.Body(body_type=pymunk.Body.STATIC)
    segment_shape = pymunk.Segment(segment_body, local_xy1,local_xy2,thickness)
    segment_body.position = global_xy
    segment_shape.friction = 0
    return segment_body, segment_shape

def trench_creator(size,trench_length, global_xy, space):
    size = int(np.ceil(size/1.5))
    segments = []
    for x in range(size):
        segment = segment_creator((x,0),(0,size-x),global_xy,1)
        segments.append(segment)

    for x in range(size):
        segment = segment_creator((size-x,0),(size,size-x),(global_xy[0]+size/2, global_xy[1]),1)
        segments.append(segment)
    for z in segments:
        for s in z:
            space.add(s)

    left_wall = segment_creator((0,0),(0,trench_length),global_xy,1)
    right_wall = segment_creator((size,0),(size,trench_length),(global_xy[0]+size/2, global_xy[1]),1)
    barrier_thickness = 100
    left_barrier = segment_creator((0,0),(0,trench_length),(global_xy[0]-barrier_thickness, global_xy[1]),barrier_thickness)
    right_barrier = segment_creator((size,0),(size,trench_length),(global_xy[0]+size/2+barrier_thickness, global_xy[1]),barrier_thickness)
    walls = [left_wall, right_wall, left_barrier, right_barrier]
    for z in walls:
        for s in z:
            space.add(s)


def get_trench_segments(space):
    """
    A function which extracts the rigid body trench objects from the pymunk space object. Space object should be passed
    from the return value of the run_simulation() function

    Returns
    -------
    List of trench segment properties, later used to draw the trench.
    """
    trench_shapes = []
    for shape, body in zip(space.shapes, space.bodies):
        if body.body_type == 2:
            trench_shapes.append(shape)

    trench_segment_props = []
    for x in trench_shapes:
        trench_segment_props.append([x.bb, x.area, x.a, x.b])

    trench_segment_props = pd.DataFrame(trench_segment_props)
    trench_segment_props.columns = ["bb", "area", "a", "b"]
    main_segments = trench_segment_props.sort_values("area", ascending=False).iloc[0:2]
    return main_segments

def wipe_space(space):
    """
    Deletes all cells in the simulation pymunk space.

    :param pymunk.Space space:
    """
    for body, poly in zip(space.bodies, space.shapes):
        if body.body_type == 0:
            space.remove(body)
            space.remove(poly)

cell_1 = Cell(length = 15*5, width = 7.5*5, y_pos = 30, x_pos = 150, angle=np.pi/2)

space = pm.Space()
options = DrawOptions()
options.collision_point_color = (1,1,0,0)
space.debug_draw(options)
space.gravity = 0, 0
cells =[cell_1]

trench_length = 150*10
ylim = trench_length
trench_creator(50, trench_length, (350, 0), space)  # Coordinates of bottom left corner of the trench


for cell in cells:
    space.add(cell.pm_object[0], cell.pm_object[1])

    
    
def space_updater(dt):
    wipe_space(space)


    for shape in space.shapes:
        if shape.body.position.y < 0 or shape.body.position.y > ylim:
            space.remove(shape.body, shape)
            space.step(dt)

    for cell in cells:
        if cell.y_pos < 0 or cell.y_pos > 600:
            cells.remove(cell)
            space.step(dt)
        elif norm.rvs() <= norm.ppf(cell.lysis_p) and len(cells) > 1:   # in case all cells disappear
            cells.remove(cell)
            space.step(dt)
        else:
            pass


    for cell in cells:
        cell.growth(np.random.uniform(low = 0, high = 0.1))
        cell.update_pm_object()
        if cell.is_dividing() == 1:
            cell.divide()
            cell.update_pm_object()
            daughter = Cell(length = cell.daughter_length(), width = cell.daughter_width(), x_pos = cell.daughter_x_pos(), y_pos = cell.daughter_y_pos(), angle=cell.daughter_angle())
            cells.append(daughter)
        cell_adder(cell)
        for x in range(150):
            space.step(dt)


def get_draw_params():
    for cell in cells:
        print(cell.y_pos)




def cell_adder(cell):
    space.add(cell.pm_object[0], cell.pm_object[1])


def update(dt):
    space_updater(dt)
    get_draw_params()


window = pyglet.window.Window(1024,768, "test", resizable = True)
options = DrawOptions()
@window.event
def on_draw():
    window.clear()
    space.debug_draw(options)

if __name__ == "__main__":
    pyglet.clock.schedule_interval(update, 1/500)
    pyglet.app.run()


30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.051064853306777
30.116783990160133
30.179085585480117
30.405402236690122
30.405402236690122
30.59338480597029
30.73667543195038
30.99006419144579
31.017681694951243
31.119792538504196
31.119792538504196
31.119792538504196
31.20918031958574
31.236915654346234
31.53096708489994
31.61069897312977
31.74478164791495
31.76017438217222
31.94164289554536
32.21570144978793
32.42487854568428
32.70163838021963
32.725665142725205
32.822153273737676
32.82252961156645
33.05138864946074
33.12446657322845
33.14033855294659
33.44848997651369
33.82528870636071
34.02469538261806
34.41904391746618
34.420717715648344
34.71850665208194
34.82

In [2]:
cells[0].pm_object[1]

Body(1e-06, 0.3221855214789193, Body.DYNAMIC)

In [3]:
space.shape_query(cells[-5].pm_object[0])

[ShapeQueryInfo(shape=<pymunk.shapes.Poly object at 0x7ff67ecd3150>, contact_point_set=ContactPointSet(normal=Vec2d(-0.3312872714211403, 0.9435299379427955), points=[ContactPoint(point_a=Vec2d(375.54641443348066, 921.5494469495724), point_b=Vec2d(375.5774396697096, 921.4610848489457), distance=-0.09365055317626148), ContactPoint(point_a=Vec2d(375.31512327487115, 921.4682372139209), point_b=Vec2d(375.348252002507, 921.3738842187207), distance=-0.10000000149011316)])),
 ShapeQueryInfo(shape=<pymunk.shapes.Poly object at 0x7ff67fdfe290>, contact_point_set=ContactPointSet(normal=Vec2d(-0.30726449017138946, -0.9516241553679247), points=[ContactPoint(point_a=Vec2d(375.74736944326537, 826.4237438563952), point_b=Vec2d(375.7780958927403, 826.5189062733498), distance=-0.10000000148989835), ContactPoint(point_a=Vec2d(376.7563478023626, 826.0979605948488), point_b=Vec2d(376.7779258962104, 826.1647897794414), distance=-0.07022644834688792)])),
 ShapeQueryInfo(shape=<pymunk.shapes.Segment object at